In diesem Beispiel wird zuerst der VKG Korpus  geladen und daraus eine Liste von Produktionen erstellt, die zur Erstellung der kontextfreien Probabilistischen Grammatik PCFG verwendet werden können. Dann wird die PCFG mit der PCFG Klasse von NLTK erstellt.

Das RNN wird mit Keras aufgebaut und besteht aus einer Einbettungsschicht, einer LSTM-Schicht und einer Dense-Schicht. Die LSTM-Schicht ermöglicht es dem Modell, die Kontextabhängigkeit zwischen den Wörtern im Korpus zu erfassen und somit eine Grammatik zu lernen, die auf den Kontext der Eingabe reagiert. Die Ausgabeschicht besteht aus einer Softmax-Aktivierungsfunktion, die eine Wahrscheinlichkeitsverteilung für die möglichen Produktionen des Startsymbols der PCFG erzeugt.

Das Modell wird mit dem Korpus trainiert, indem die Eingabe eine Sequenz von Wortindizes ist und die Ausgabe eine zufällige Produktion der PCFG für den aktuellen Satz im Korpus ist. Nach dem Training wird das Modell Korpus getestet und die Ausgabe als eine der möglichen Produktionen des Startsymbols der PCFG interpretiert.

Die Ausgabe des Modells ist eine zufällige Produktion aus der PCFG, die dem eingegebenen Satz entspricht.


In [ ]:
import numpy as np
import nltk
from nltk import PCFG
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Einlesen des Korpus
f = open('VKGKORPUS.TXT','r')
corpus = f.read()
f.close()

# Erstellen der Produktionen
productions = []
for tree in corpus:
    productions += tree.productions()

# Erstellen der PCFG
pcfg = PCFG(productions)

# Erstellen des RNN
vocab = list(set([word for sent in corpus for word in sent.leaves()]))
vocab_size = len(vocab)
embedding_size = 50
hidden_size = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=None))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dense(pcfg.start().symbol().split()[0].count('|')+1, activation='softmax'))

# Kompilieren und Trainieren des Modells
model.compile(loss='categorical_crossentropy', optimizer='adam')
X = np.array([list(map(vocab.index, sent.leaves())) for sent in corpus])
Y = np.array([pcfg.generate() for i in range(len(corpus))])
model.fit(X, Y, epochs=10, batch_size=32)

# Testen des Modells
test_sentence = corpus
test_input = np.array([[vocab.index(word) for word in test_sentence.split()]])
test_output = model.predict(test_input)[0]

# Ausgabe der PCFG / kontextfreien Grammatik
print(PCFG.fromstring(str(pcfg.start()).replace('|', '').replace('->', '').replace(' ', '\n')+'\n').productions()[test_output.argmax()])
